In [268]:
import pandas as pd

pd.set_option('display.max_rows', 10)

# Create an empty DataFrame with the specified columns
df = pd.read_csv("data50.csv", index_col=0)
df2 = pd.read_csv("data20.csv", index_col=0)

# Show the empty DataFrame
data = df


In [253]:
# Replace NaN values with 0
data_filled = data.fillna(0)

# Ensure all columns except 'replicates' (since it might contain non-numeric values originally) are numeric
data_filled = data_filled.apply(pd.to_numeric, errors='ignore')

# Display the data types to confirm changes
data_filled.dtypes, data_filled.head()

data_filled


,round,replicates,duplicates,distortion,m1,m2,m3,date
0,-1,_21,100.0,1,1.000000,0.794769,0.885650,Fri_Feb__9_08_51_26_2024
1,-1,_21,8.0,1,1.000000,0.688249,0.815341,Fri_Feb__9_08_51_30_2024
2,-1,_21,1.0,0,0.997807,0.919192,0.956888,Tue_Feb__6_20_11_45_2024
3,-1,_21,0.1,0,0.967742,0.952381,0.960000,Tue_Feb__6_20_11_41_2024
4,-1,_21,100.0,0,1.000000,0.921762,0.959288,Tue_Feb__6_20_11_52_2024
...,...,...,...,...,...,...,...,...
2911,-12,_50,100.0,1,0.997545,0.817572,0.898636,Thu_Feb_29_00_22_02_2024
2912,-12,_50,0.1,1,1.000000,0.783333,0.878505,Thu_Feb_29_00_21_57_2024
2913,-12,_50,1.0,1,0.987113,0.786448,0.875429,Thu_Feb_29_15_18_15_2024
2914,-12,_50,0.1,0,0.967742,0.952381,0.960000,Thu_Feb_29_00_21_41_2024


In [254]:
# Remove "_" from the 'replicates' column and convert to numeric if possible
data_filled['replicates'] = data_filled['replicates'].astype(str).str.replace("_", "")
data_filled['replicates'] = pd.to_numeric(data_filled['replicates'], errors='coerce').fillna(0)

# Make 'round' and 'split' values positive
data_filled['round'] = data_filled['round'].abs()


# Rename columns 'm1', 'm2', 'm3' to 'precision', 'recall', 'f1score'
data_renamed = data_filled.rename(columns={'m1': 'precision', 'm2': 'recall', 'm3': 'f1score'})

# Display the first few rows to confirm the column names have been updated
data_renamed


,round,replicates,duplicates,distortion,precision,recall,f1score,date
0,1,21,100.0,1,1.000000,0.794769,0.885650,Fri_Feb__9_08_51_26_2024
1,1,21,8.0,1,1.000000,0.688249,0.815341,Fri_Feb__9_08_51_30_2024
2,1,21,1.0,0,0.997807,0.919192,0.956888,Tue_Feb__6_20_11_45_2024
3,1,21,0.1,0,0.967742,0.952381,0.960000,Tue_Feb__6_20_11_41_2024
4,1,21,100.0,0,1.000000,0.921762,0.959288,Tue_Feb__6_20_11_52_2024
...,...,...,...,...,...,...,...,...
2911,12,50,100.0,1,0.997545,0.817572,0.898636,Thu_Feb_29_00_22_02_2024
2912,12,50,0.1,1,1.000000,0.783333,0.878505,Thu_Feb_29_00_21_57_2024
2913,12,50,1.0,1,0.987113,0.786448,0.875429,Thu_Feb_29_15_18_15_2024
2914,12,50,0.1,0,0.967742,0.952381,0.960000,Thu_Feb_29_00_21_41_2024


In [255]:
# Define a function to map 'round' values to model names
def map_round_to_model(round_number):
    if 1 <= round_number <= 3:
        return "Both"
    elif 4 <= round_number <= 6:
        return "No Diri"
    elif 7 <= round_number <= 9:
        return "None"
    elif 10 <= round_number <= 12:
        return "No Empirical"
    else:
        return "Unknown"  # For any round numbers outside the specified ranges

# Apply the mapping function to the 'round' column to create the new 'model' column
data_renamed['model'] = data_renamed['round'].apply(map_round_to_model)



# Define a function to map 'round' values to prior categories
def map_round_to_prior(round_number):
    if round_number in [1, 4, 7, 10]:
        return "Pitman"
    elif round_number in [2, 5, 8, 11]:
        return "uniform"
    elif round_number in [3, 6, 9, 12]:
        return "Bounded NBD"
    else:
        return "Unknown"  # For any round numbers outside the specified or considered ranges

# Apply the mapping function to the 'round' column to create the new 'prior' column
data_renamed['prior'] = data_renamed['round'].apply(map_round_to_prior)

# Define a function to map 'duplicates' values to duplicates level categories
def map_duplicates_to_level(duplicates_value):
    if duplicates_value == 0.1:
        return "Low"
    elif duplicates_value == 1.0:
        return "Medium"
    elif duplicates_value == 8.0:
        return "High"
    elif duplicates_value == 100.0:
        return "Very High"
    else:
        return "Unknown"  # For any duplicates values outside the specified ranges

# Apply the mapping function to the 'duplicates' column to create the new 'duplicates_level' column
data_renamed['duplicates_level'] = data_renamed['duplicates'].apply(map_duplicates_to_level)

# Define a function to map 'distortion' values to distortion level categories
def map_distortion_to_level(distortion_value):
    if distortion_value == 0:
        return "Low"
    elif distortion_value == 1:
        return "High"
    else:
        return "Unknown"  # For any distortion values outside the specified values

# Apply the mapping function to the 'distortion' column to create the new 'distortion_level' column
data_renamed['distortion_level'] = data_renamed['distortion'].apply(map_distortion_to_level)

def map_iter_to_num(iter):
    if iter == '10k':
        return 10000
    elif iter == '50k':
        return 50000
    else:
        return 0  # For any distortion values outside the specified values


# Select only the specified columns
data_selected_columns = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates', "date"]]

# Select and reorder the dataset with only the specified columns
data_final = data_renamed[['model', 'prior', 'duplicates_level', 'distortion_level',
                           'replicates', 'precision', 'recall', 'f1score', "date"]]

# Display the first few rows to confirm the changes
data_final






,model,prior,duplicates_level,distortion_level,replicates,precision,recall,f1score,date
0,Both,Pitman,Very High,High,21,1.000000,0.794769,0.885650,Fri_Feb__9_08_51_26_2024
1,Both,Pitman,High,High,21,1.000000,0.688249,0.815341,Fri_Feb__9_08_51_30_2024
2,Both,Pitman,Medium,Low,21,0.997807,0.919192,0.956888,Tue_Feb__6_20_11_45_2024
3,Both,Pitman,Low,Low,21,0.967742,0.952381,0.960000,Tue_Feb__6_20_11_41_2024
4,Both,Pitman,Very High,Low,21,1.000000,0.921762,0.959288,Tue_Feb__6_20_11_52_2024
...,...,...,...,...,...,...,...,...,...
2911,No Empirical,Bounded NBD,Very High,High,50,0.997545,0.817572,0.898636,Thu_Feb_29_00_22_02_2024
2912,No Empirical,Bounded NBD,Low,High,50,1.000000,0.783333,0.878505,Thu_Feb_29_00_21_57_2024
2913,No Empirical,Bounded NBD,Medium,High,50,0.987113,0.786448,0.875429,Thu_Feb_29_15_18_15_2024
2914,No Empirical,Bounded NBD,Low,Low,50,0.967742,0.952381,0.960000,Thu_Feb_29_00_21_41_2024


In [256]:
data_cleaned = data_final

# Finding duplicates and keeping them to show
duplicates_to_show = data_cleaned[data_cleaned.duplicated(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], keep='first')]
duplicates_to_show2 = data_cleaned[data_cleaned.duplicated(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], keep='last')]

duplicates_to_show3 = pd.concat([duplicates_to_show, duplicates_to_show2])

# Sorting the duplicates for better readability
duplicates_to_show3 = duplicates_to_show3.sort_values(by=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'])

pd.set_option('display.max_rows', 10)

duplicates_to_show3 = duplicates_to_show3[(duplicates_to_show3['precision'] >= 0.3) & (duplicates_to_show3['recall'] >= 0.3)]

data_cleaned.drop_duplicates(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], keep=False)

# duplicates_to_show3


,model,prior,duplicates_level,distortion_level,replicates,precision,recall,f1score,date
0,Both,Pitman,Very High,High,21,1.000000,0.794769,0.885650,Fri_Feb__9_08_51_26_2024
1,Both,Pitman,High,High,21,1.000000,0.688249,0.815341,Fri_Feb__9_08_51_30_2024
2,Both,Pitman,Medium,Low,21,0.997807,0.919192,0.956888,Tue_Feb__6_20_11_45_2024
3,Both,Pitman,Low,Low,21,0.967742,0.952381,0.960000,Tue_Feb__6_20_11_41_2024
4,Both,Pitman,Very High,Low,21,1.000000,0.921762,0.959288,Tue_Feb__6_20_11_52_2024
...,...,...,...,...,...,...,...,...,...
2911,No Empirical,Bounded NBD,Very High,High,50,0.997545,0.817572,0.898636,Thu_Feb_29_00_22_02_2024
2912,No Empirical,Bounded NBD,Low,High,50,1.000000,0.783333,0.878505,Thu_Feb_29_00_21_57_2024
2913,No Empirical,Bounded NBD,Medium,High,50,0.987113,0.786448,0.875429,Thu_Feb_29_15_18_15_2024
2914,No Empirical,Bounded NBD,Low,Low,50,0.967742,0.952381,0.960000,Thu_Feb_29_00_21_41_2024


In [257]:
data_cleaned[(data_cleaned['precision'] <= 0.3) | (data_cleaned['recall'] <= 0.3)]

,model,prior,duplicates_level,distortion_level,replicates,precision,recall,f1score,date
41,Both,Pitman,Very High,High,25,0.002955,1.000000,0.005893,Wed_Feb_28_11_14_53_2024
43,Both,Pitman,Low,High,25,0.000123,1.000000,0.000245,Wed_Feb_28_11_14_46_2024
48,Both,Pitman,Medium,High,25,0.000975,1.000000,0.001948,Wed_Feb_28_11_14_48_2024
51,Both,Pitman,High,High,25,0.002608,1.000000,0.005202,Wed_Feb_28_11_14_51_2024
54,Both,Pitman,High,High,26,0.002608,1.000000,0.005202,Wed_Feb_28_11_15_31_2024
...,...,...,...,...,...,...,...,...,...
1925,None,uniform,Low,High,47,0.258883,0.850000,0.396887,Wed_Feb_28_23_05_35_2024
1935,None,uniform,Low,High,48,0.265625,0.850000,0.404762,Wed_Feb_28_23_06_36_2024
1940,None,uniform,Low,High,49,0.274193,0.850000,0.414634,Wed_Feb_28_23_07_00_2024
1943,None,uniform,Low,Low,49,0.298507,0.952381,0.454545,Wed_Feb_28_23_06_43_2024


In [258]:
data_cleaned = data_cleaned[(data_cleaned['precision'] >= 0.2) & (data_cleaned['recall'] >= 0.2)]
data_cleaned = data_cleaned.drop_duplicates(subset=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], keep="first")
data_cleaned.reset_index(drop=True, inplace=True)
data_cleaned.shape

(2880, 9)

In [259]:
# # Select the columns of interest
# columns_of_interest = ['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates']
# df_selected = data_cleaned[columns_of_interest]


# # Option 2: Count across selected columns (if you want a total count regardless of the column)
# total_count = df_selected[['model', 'prior', 'replicates']].value_counts() # Sorting by index (the number) for better readability
# print("Total count across selected columns:")
# print(total_count)

In [260]:
# Checking for missing combinations in the first 6 columns
# First, we create a DataFrame of all possible combinations of these columns

# Get unique values for each of the first 6 columns
model_values = data_cleaned['model'].unique()
prior_values = data_cleaned['prior'].unique()
duplicates_level_values = data_cleaned['duplicates_level'].unique()
distortion_level_values = data_cleaned['distortion_level'].unique()
replicates_values = data_cleaned['replicates'].unique()

# Use itertools.product to create all possible combinations
from itertools import product
all_combinations = list(product(model_values, prior_values, duplicates_level_values, distortion_level_values, replicates_values))

# Convert to DataFrame
all_combinations_df = pd.DataFrame(all_combinations, columns=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'])

# Now, find missing combinations by checking which rows in all_combinations_df are not present in data_cleaned_no_duplicates
missing_combinations = pd.merge(all_combinations_df, data_cleaned, on=['model', 'prior', 'duplicates_level', 'distortion_level', 'replicates'], how='left', indicator=True).query('_merge == "left_only"').drop(columns=['precision', 'recall', 'f1score', '_merge'])

missing_combinations_count = missing_combinations.shape[0]

print(missing_combinations_count)
pd.set_option('display.max_rows', None)
missing_combinations


0


,model,prior,duplicates_level,distortion_level,replicates,date


In [261]:
pd.set_option('display.max_rows', 10)

In [262]:
# Group by the specified columns and compute mean and standard deviation for precision, recall, and f1score
grouped_data = data_final.groupby(['model', 'prior', 'duplicates_level', 'distortion_level'])

# Compute mean
mean_values = grouped_data[['precision', 'recall', 'f1score']].mean().reset_index()

# Compute standard deviation
std_dev_values = grouped_data[['precision', 'recall', 'f1score']].std().reset_index()

# Rename columns for clarity
mean_values.rename(columns={
    'precision': 'precision_mean', 
    'recall': 'recall_mean', 
    'f1score': 'f1score_mean'}, inplace=True)

std_dev_values.rename(columns={
    'precision': 'precision_std', 
    'recall': 'recall_std', 
    'f1score': 'f1score_std'}, inplace=True)

# Display the mean values
mean_values.head(), std_dev_values.head()


(  model        prior duplicates_level distortion_level  precision_mean  \
 0  Both  Bounded NBD             High             High        0.999016   
 1  Both  Bounded NBD             High              Low        1.000000   
 2  Both  Bounded NBD              Low             High        1.000000   
 3  Both  Bounded NBD              Low              Low        0.979410   
 4  Both  Bounded NBD           Medium             High        0.989773   
 
    recall_mean  f1score_mean  
 0     0.712005      0.831101  
 1     0.936635      0.967280  
 2     0.747396      0.855419  
 3     0.952381      0.965686  
 4     0.738835      0.846087  ,
   model        prior duplicates_level distortion_level  precision_std  \
 0  Both  Bounded NBD             High             High       0.001474   
 1  Both  Bounded NBD             High              Low       0.000000   
 2  Both  Bounded NBD              Low             High       0.000000   
 3  Both  Bounded NBD              Low              Low    

In [263]:
mean_values.shape

(96, 7)

In [264]:
# Merge mean and standard deviation dataframes on the groupby columns
combined_data = pd.merge(mean_values, std_dev_values, 
                         on=['model', 'prior', 'duplicates_level', 'distortion_level'])


pd.set_option('display.max_rows', None)


# Display the combined dataframe
combined_data


,model,prior,duplicates_level,distortion_level,precision_mean,recall_mean,f1score_mean,precision_std,recall_std,f1score_std
0,Both,Bounded NBD,High,High,0.999016,0.712005,0.831101,0.001474,0.029264,0.019378
1,Both,Bounded NBD,High,Low,1.000000,0.936635,0.967280,0.000000,0.001412,0.000753
2,Both,Bounded NBD,Low,High,1.000000,0.747396,0.855419,0.000000,0.007465,0.004905
3,Both,Bounded NBD,Low,Low,0.979410,0.952381,0.965686,0.009243,0.000000,0.004513
4,Both,Bounded NBD,Medium,High,0.989773,0.738835,0.846087,0.001236,0.002358,0.001474
5,Both,Bounded NBD,Medium,Low,0.999854,0.923434,0.960124,0.000554,0.002970,0.001628
6,Both,Bounded NBD,Very High,High,0.998032,0.795691,0.885236,0.001060,0.025260,0.015436
7,Both,Bounded NBD,Very High,Low,1.000000,0.921521,0.959157,0.000000,0.001164,0.000631
8,Both,Pitman,High,High,0.941330,0.710584,0.770267,0.238210,0.077150,0.194797
9,Both,Pitman,High,Low,1.000000,0.936901,0.967422,0.000000,0.001570,0.000836


In [265]:
pd.set_option('display.max_rows', 10)

In [266]:
combined_data.to_csv("clean_data.csv")